# HIERACHICAL CLUSTERING

Alla base di questo clusetring troviamo la seguente logica:
Una matrice delle distanze tra i punti (le righe del dataset), questa utilizza una distanza prefissata, comunemente quella euclidea. L'obbiettivo è quello di ottenere un clustering totale tra tutti i punti, tenendo traccia della sequenza delle unioni fatte durante l'algorimo. Ad ogni step vengono difatti creati i cluster con i punti più vicini, poi questi cluster vengono uniti ad altri e così via.
La differenza tra le tipologie di hierarchical clustering risiede nella logica con cui uniamo due cluster, quindi nella distanza che prendiamo in considerazione per scegliere i cluster più vicini da unire nello step.

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Esempio di dataset
data = {
    'height': [180, 175, 160, 170, 165],
    'weight': [80, 70, 55, 65, 60],
    'bmi': [24.7, 22.9, 21.5, 22.5, 22.0]
}
df = pd.DataFrame(data)

# Standardizza le feature
scaler = StandardScaler()
X = scaler.fit_transform(df)